# 1.6 - 1.7. Mô tả và đánh giá dữ liệu & Tiền xử lí dữ liệu

* Load toàn bộ review vào một dataframe duy nhất.

In [1]:
%load_ext autoreload
%autoreload 2

In [21]:
!pip3 install pyenchant

In [84]:
!pip3 list

Package                 Version
----------------------- ---------
absl-py                 0.15.0
appnope                 0.1.3
argon2-cffi             21.3.0
argon2-cffi-bindings    21.2.0
astunparse              1.6.3
async-generator         1.10
attrs                   21.4.0
autopep8                2.0.1
backcall                0.2.0
bleach                  4.1.0
cached-property         1.5.2
cachetools              4.2.4
catboost                1.0.6
certifi                 2022.12.7
cffi                    1.15.0
charset-normalizer      2.0.12
clang                   5.0
cycler                  0.11.0
dataclasses             0.8
decorator               4.4.2
defusedxml              0.7.1
emojis                  0.7.0
entrypoints             0.4
et-xmlfile              1.1.0
flatbuffers             1.12
gast                    0.4.0
gensim                  4.2.0
google-auth             1.35.0
google-auth-oauthlib    0.4.6
google-pasta            0.2.0
graphviz                0.19.1

In [92]:
import modules.utils as Utils
import modules.processor as Processor
import numpy as np
import pandas as pd
import enchant
import random

from sklearn.utils import shuffle

In [29]:
# Lấy tất cả các directory path của các lần ta tiến hành crawl data
dir_paths = Utils.getAllFolderPath("./data/product_reviews/")

dir_paths

['./data/product_reviews/product_reviews_02/',
 './data/product_reviews/product_reviews_03/',
 './data/product_reviews/product_reviews_01/',
 './data/product_reviews/product_reviews_00/']

In [43]:
# Đọc toàn bộ các review từ các file csv
reviews = Utils.readReviews(dir_paths)

In [44]:
reviews.tail()

,raw_comment,rating
278154,"Hàng ok lắm có điều mùi nhựa nồng quá, trc khi...",5
278155,Mua size L hơi to nhưng vẫn ok,5
278156,"Giá tốt. Chất lượng chắc chắn, thiết kế rất đẹ...",5
278157,Sản phẩm tốt dày chất lượng size m nhung to lắ...,5
278158,"Đẹp lắm ạ\nCảm ơn shop, có điều bên ngoài dính...",5


In [45]:
print("Tập dữ liệu có {} bình luận.".format(reviews.shape[0]))

Tập dữ liệu có 278159 bình luận.


* Đếm tần số xuất hiện của từng rating.

In [46]:
reviews['rating'].value_counts()

5    260555
4      9646
3      4194
1      2308
2      1456
Name: rating, dtype: int64

> **Nhận xét**:
> * Nhìn chung tuy ta crawl được hơn 200,000 quan sát nhưng có sự chênh lệch lớn giữa các rating.
> * Nhìn qua ta thấy đa phần là các rating được đánh giá 5 sao, điều này cũng dễ hiểu vì hệ thống recommend của Shopee sẽ ưu tiên gợi ý cho khách hàng những sản phẩm có đánh giá tốt. Và sẽ hạn chế hoặc thậm chí là ko gợi ý các mặc hàng bị đánh giá kém. Nên với địa vị là người đi trộm dữ liệu như chúng ta thì ko có cách nào khắc phục điều này.
> * Bây giờ, do ta cần chi ra hai lớp là negative và positive nên những comment mà `rating` $\geq 4$ sẽ được cho vào nhóm positive, ngược lại là nhóm negative. 

* Tiến hành label cho `reviews` với các giá `rating` $< 4$ sẽ thuộc nhóm negative còn lại là nhóm positive.

In [47]:
reviews = Utils.labelRating(reviews)

In [48]:
reviews.tail()

,raw_comment,rating,label
278154,"Hàng ok lắm có điều mùi nhựa nồng quá, trc khi...",5,1
278155,Mua size L hơi to nhưng vẫn ok,5,1
278156,"Giá tốt. Chất lượng chắc chắn, thiết kế rất đẹ...",5,1
278157,Sản phẩm tốt dày chất lượng size m nhung to lắ...,5,1
278158,"Đẹp lắm ạ\nCảm ơn shop, có điều bên ngoài dính...",5,1


* Bây giờ ta sẽ xóa đi feature `rating` vì về sau ta sẽ ko cần dúng đến nó nữa, và tiến hành đếm số lượng quan sát của từng nhóm trên `label`

In [49]:
reviews = reviews.drop(columns=['rating'])

Processor.printAfterProcess(reviews)
reviews.tail()

Shape: (278159, 2)
1    270201
0      7958
Name: label, dtype: int64


,raw_comment,label
278154,"Hàng ok lắm có điều mùi nhựa nồng quá, trc khi...",1
278155,Mua size L hơi to nhưng vẫn ok,1
278156,"Giá tốt. Chất lượng chắc chắn, thiết kế rất đẹ...",1
278157,Sản phẩm tốt dày chất lượng size m nhung to lắ...,1
278158,"Đẹp lắm ạ\nCảm ơn shop, có điều bên ngoài dính...",1


<hr>

* Một trong những vấn đề đầu tiên và tối quan trong khi xử lí với dữ liệu văn bản là kiểm tra xem liệu text `a` có cùng cách biểu diễn với text `b` hay không.
* Một ví dụ dễ hiểu là giả sử ta có biến `a = 'đẹp'` và biến `b = 'đep'`, nhưng khi ta compare hai biến này `a == b` thì kết quả sẽ ra False, nguyên nhân là do chúng sử dụng mã hóa unicode khác nhau, có thể `a` dùng unicode-8 và `b` dùng unicode-16.
* Vậy điều đầu tiên ta cần làm là phải đưa tất cả các text về cùng một **chuẩn** duy nhất, ta có thể làm điều này bằng cách sử dụng `unicodedata.normalize()` từ package chuẩn `unicodedata` của Python. _(tham khảo thêm tại đây [https://www.kite.com/python/docs/unicodedata.normalize](https://www.kite.com/python/docs/unicodedata.normalize))_
* Dưới đây là ví dụ cho trường hợp này:

In [50]:
import unicodedata

a = 'đẹp, rất hài lòng'
b = 'đẹp, rất hài lòng'

a == b

False

In [51]:
''' Sử dụng chuẩn NFD '''
a = unicodedata.normalize('NFD', a)
b = unicodedata.normalize('NFD', b)

print("String a: {}, Type a: {}".format(a, type(a)))
print("String b: {}, Type b: {}".format(b, type(b)))

String a: đẹp, rất hài lòng, Type a: <class 'str'>
String b: đẹp, rất hài lòng, Type b: <class 'str'>


In [52]:
a == b

True

* Bây giờ, ta sẽ tạo một feature có tên là `normalize_comment`, trải qua 2 bước:
  * `lower()` cho text.
  * Chuẩn hóa bằng `unicodedata.normalize()`

In [53]:
reviews['normalize_comment'] = reviews['raw_comment'].apply(lambda cmt: Processor.normalizeComment(cmt))

reviews.tail()

,raw_comment,label,normalize_comment
278154,"Hàng ok lắm có điều mùi nhựa nồng quá, trc khi...",1,hàng ok lắm có điều mùi nhựa nồng q...
278155,Mua size L hơi to nhưng vẫn ok,1,mua size l hơi to nhưng vẫn ok
278156,"Giá tốt. Chất lượng chắc chắn, thiết kế rất đẹ...",1,"giá tốt. chất lượng chắc chắn, thi..."
278157,Sản phẩm tốt dày chất lượng size m nhung to lắ...,1,sản phẩm tốt dày chất lượng size m ...
278158,"Đẹp lắm ạ\nCảm ơn shop, có điều bên ngoài dính...",1,"đẹp lắm ạ\ncảm ơn shop, có điều bên ..."


* Ta cũng sẽ chuẩn hóa cho cột `raw_comment` nhưng không `lower()` chúng.

In [54]:
reviews['raw_comment'] = reviews['raw_comment'].apply(lambda cmt: Processor.normalizeComment(cmt, False))

reviews.tail()

,raw_comment,label,normalize_comment
278154,Hàng ok lắm có điều mùi nhựa nồng q...,1,hàng ok lắm có điều mùi nhựa nồng q...
278155,Mua size L hơi to nhưng vẫn ok,1,mua size l hơi to nhưng vẫn ok
278156,"Giá tốt. Chất lượng chắc chắn, thi...",1,"giá tốt. chất lượng chắc chắn, thi..."
278157,Sản phẩm tốt dày chất lượng size m ...,1,sản phẩm tốt dày chất lượng size m ...
278158,"Đẹp lắm ạ\nCảm ơn shop, có điều bên ...",1,"đẹp lắm ạ\ncảm ơn shop, có điều bên ..."


<hr>

* Chúng ta biết rằng, các comment của các sản phẩm đôi khi sẽ chứa các URL do người bán hàng chèn vào để giúp khách hàng có thể click vào để xem các mặt hàng khác, chúng là các noise sample mà ta cần phải loại bỏ khỏi dataset của chúng ta.
* Hình dưới đây là kết quả cho ra khi ta thử search cụm từ `http` thì nó cho ra hơn 600 mẫu dữ liệu chứa URL. Ta cần loại bỏ các mẫu này.<br>
  ![](./images/04.png)

In [57]:
reviews['contain_url'] = reviews['normalize_comment'].apply(lambda cmt: Processor.containsURL(cmt))

Processor.printAfterProcess(reviews, 'contain_url')
reviews.tail()

Shape: (277589, 4)
0    277589
Name: contain_url, dtype: int64


,raw_comment,label,normalize_comment,contain_url
277584,Hàng ok lắm có điều mùi nhựa nồng q...,1,hàng ok lắm có điều mùi nhựa nồng q...,0
277585,Mua size L hơi to nhưng vẫn ok,1,mua size l hơi to nhưng vẫn ok,0
277586,"Giá tốt. Chất lượng chắc chắn, thi...",1,"giá tốt. chất lượng chắc chắn, thi...",0
277587,Sản phẩm tốt dày chất lượng size m ...,1,sản phẩm tốt dày chất lượng size m ...,0
277588,"Đẹp lắm ạ\nCảm ơn shop, có điều bên ...",1,"đẹp lắm ạ\ncảm ơn shop, có điều bên ...",0


In [58]:
reviews = reviews[reviews['contain_url'] == 0]
reviews = reviews.drop(columns=['contain_url']).reset_index(drop=True) # xóa cột `contain_url`

Processor.printAfterProcess(reviews)
reviews.tail()

Shape: (277589, 3)
1    269633
0      7956
Name: label, dtype: int64


,raw_comment,label,normalize_comment
277584,Hàng ok lắm có điều mùi nhựa nồng q...,1,hàng ok lắm có điều mùi nhựa nồng q...
277585,Mua size L hơi to nhưng vẫn ok,1,mua size l hơi to nhưng vẫn ok
277586,"Giá tốt. Chất lượng chắc chắn, thi...",1,"giá tốt. chất lượng chắc chắn, thi..."
277587,Sản phẩm tốt dày chất lượng size m ...,1,sản phẩm tốt dày chất lượng size m ...
277588,"Đẹp lắm ạ\nCảm ơn shop, có điều bên ...",1,"đẹp lắm ạ\ncảm ơn shop, có điều bên ..."


> **Nhận xét**
> * Đa phần là các bình luận thuộc nhóm positive sẽ chứa các URL, cũng dễ hiểu vì họ quảng cáo mà 😅.

<hr>

* Tiếp theo, ta cũng cần xóa các comment mà chữ IN HOA chiếm quá 50% độ dài comment, các comment này khả năng cao cũng là quảng cáo, vì người bán họ muốn làm nổi bật bình luận này lên so với các bình luận còn lại.

In [60]:
reviews['contain_adv'] = reviews['raw_comment'].apply(lambda cmt: Processor.containAdvertisement(cmt))

Processor.printAfterProcess(reviews, 'contain_adv')
reviews.tail()

Shape: (277589, 4)
0    270936
1      6653
Name: contain_adv, dtype: int64


,raw_comment,label,normalize_comment,contain_adv
277584,Hàng ok lắm có điều mùi nhựa nồng q...,1,hàng ok lắm có điều mùi nhựa nồng q...,0
277585,Mua size L hơi to nhưng vẫn ok,1,mua size l hơi to nhưng vẫn ok,0
277586,"Giá tốt. Chất lượng chắc chắn, thi...",1,"giá tốt. chất lượng chắc chắn, thi...",0
277587,Sản phẩm tốt dày chất lượng size m ...,1,sản phẩm tốt dày chất lượng size m ...,0
277588,"Đẹp lắm ạ\nCảm ơn shop, có điều bên ...",1,"đẹp lắm ạ\ncảm ơn shop, có điều bên ...",0


> **Nhận xét**
> * Các mẫu tìm năng chứa quảng cáo khá cao, lên đến hơn 3000 sample. Ta có thể xóa chúng.

In [61]:
reviews = reviews[reviews['contain_adv'] == 0]
reviews = reviews.drop(columns=['contain_adv']).reset_index(drop=True) # xóa cột `contain_adv`

Processor.printAfterProcess(reviews)
reviews.tail()

Shape: (270936, 3)
1    263084
0      7852
Name: label, dtype: int64


,raw_comment,label,normalize_comment
270931,Hàng ok lắm có điều mùi nhựa nồng q...,1,hàng ok lắm có điều mùi nhựa nồng q...
270932,Mua size L hơi to nhưng vẫn ok,1,mua size l hơi to nhưng vẫn ok
270933,"Giá tốt. Chất lượng chắc chắn, thi...",1,"giá tốt. chất lượng chắc chắn, thi..."
270934,Sản phẩm tốt dày chất lượng size m ...,1,sản phẩm tốt dày chất lượng size m ...
270935,"Đẹp lắm ạ\nCảm ơn shop, có điều bên ...",1,"đẹp lắm ạ\ncảm ơn shop, có điều bên ..."


> **Nhận xét**
> * Lại một lần nữa các comment có khả năng cao là quảng cáo này lại đa phần là thuộc nhóm positive.

<hr>

* Nhìn qua các comment, ta sẽ thấy có các comment chứa emoji như hình dưới đây:<br>
  ![](./images/05.png)

* Đây là "vốn quý" góp phần làm tăng sức mạnh cho model, nếu ta thực hiện bước loại bỏ các kí tự đặc biệt trước khi ta tách các emoji ra, thì ta đã vô tình xóa luôn các emoji này, vì các emoji thực chất được xây dựng dựa trên các kí tự đặc biệt.
* Như hình trên, rõ ràng ta thấy được emoji góp phần ta hiểu được một comment là positive hay negative.
* Ta sẽ sử dụng một gói của python là `emojis`:
  ```shell
  pip3 install emojis
  ```
  gói này sẽ giúp ta tách các emoji ra khỏi bình luận.
* Ta sẽ chứa toàn bộ emoji của một comment qua cột tương ứng là `emoji`

In [63]:
reviews['emoji'] = reviews['raw_comment'].apply(lambda cmt: Processor.extractEmoji(cmt))

reviews.tail()

,raw_comment,label,normalize_comment,emoji
270931,Hàng ok lắm có điều mùi nhựa nồng q...,1,hàng ok lắm có điều mùi nhựa nồng q...,
270932,Mua size L hơi to nhưng vẫn ok,1,mua size l hơi to nhưng vẫn ok,
270933,"Giá tốt. Chất lượng chắc chắn, thi...",1,"giá tốt. chất lượng chắc chắn, thi...",
270934,Sản phẩm tốt dày chất lượng size m ...,1,sản phẩm tốt dày chất lượng size m ...,
270935,"Đẹp lắm ạ\nCảm ơn shop, có điều bên ...",1,"đẹp lắm ạ\ncảm ơn shop, có điều bên ...",


In [64]:
",".join(reviews['emoji'])[:200]

',,,,,,,,,,😻,,,,👍,,,,,,,✨,,,,,,,,,,,,,😜,,,,,,,,,❤️,,🤣,,,👍,,,,😘,,😘,,,,,,,,,,,,,👍 ❤️,,👍 ❤️,,,❤️,\U0001f971,🎉,,❤️,\U0001f971,🎉,,,,,,,,,❤️,,,😅,,,💜 \U0001f970 🖤,,,💜 \U0001f970 🖤,,🙆,,🙆,,,,,,,,,,,,\U0001f97a,,,,,,,,,,,,,,,😊 😅 ❤️ 😀 \U0001f92a,,,,,♂️ 🤦 🙏,,,,,,,,,,'

<hr>

* Tiếp theo, ta sẽ loại bỏ dấu câu, kí tự đặc biệt

In [65]:
reviews['normalize_comment'] = reviews['normalize_comment'].apply(lambda cmt: Processor.removeSpecialLetters(cmt.lower()))

reviews.tail()

,raw_comment,label,normalize_comment,emoji
270931,Hàng ok lắm có điều mùi nhựa nồng q...,1,hàng ok lắm có điều mùi nhựa nồng q...,
270932,Mua size L hơi to nhưng vẫn ok,1,mua size l hơi to nhưng vẫn ok,
270933,"Giá tốt. Chất lượng chắc chắn, thi...",1,giá tốt chất lượng chắc chắn thiê...,
270934,Sản phẩm tốt dày chất lượng size m ...,1,sản phẩm tốt dày chất lượng size m ...,
270935,"Đẹp lắm ạ\nCảm ơn shop, có điều bên ...",1,đẹp lắm ạ cảm ơn shop có điều bên ng...,


<hr>

* Tiếp theo, ta cần chuẩn lại các từ bị dupplicate như: _chờiiiiiii ơiiiiiii, xinhhhhhhh quá, đẹp xỉuuuuuuuuuuuu_ thành _chời ơi, xinh quá, đẹp xỉu_.
* Tuy nhiên có một vấn đề xảy ra, giả sử trong comment có các từ tiếng anh như "_feedback_", thì nó sẽ thành "_fedback_", nên ta sẽ thực hiện bước này ở phần sau:

```python
reviews['normalize_comment'] = reviews['normalize_comment'].apply(lambda cmt: Processor.removeDuplicateLetters(cmt))

```

<hr>

* Tiếp theo, chúng ta sẽ chuẩn lại một vài từ viết tắt cơ bản.
* File `modules/dependencies/abbreviate.txt` chứa các từ viết tắt cơ bản mà giới trẻ hay dùng comment, ta có thể bổ sung theo thời gian.

In [68]:
# xây dựng dictionary cho các từ viết tắt
abbreviate = Utils.buildDictionaryFromFile("./modules/dependencies/abbreviate.txt")

# test
abbreviate['okela']

'ok'

In [69]:
reviews['normalize_comment'] = reviews['normalize_comment'].apply(lambda cmt: Processor.replaceWithDictionary(cmt, abbreviate))

reviews.tail()

,raw_comment,label,normalize_comment,emoji
270931,Hàng ok lắm có điều mùi nhựa nồng q...,1,hàng ok lắm có điều mùi nhựa nồng q...,
270932,Mua size L hơi to nhưng vẫn ok,1,mua size l hơi to nhưng vẫn ok,
270933,"Giá tốt. Chất lượng chắc chắn, thi...",1,giá tốt chất lượng chắc chắn thiê...,
270934,Sản phẩm tốt dày chất lượng size m ...,1,sản phẩm tốt dày chất lượng size mi...,
270935,"Đẹp lắm ạ\nCảm ơn shop, có điều bên ...",1,đẹp lắm ạ cảm ơn shop có điều bên ng...,


<hr>

* Bây giờ ta sẽ tiến hành xóa các từ vô nghĩa trong comment, ví dụ như hình dưới đây:<br>
  ![](./images/06.png)
  những bình luận này mặc dù thuộc lớp positive nhưng nó là các noise sample, có thể các comment này dùng để comment cho có để nhận shopee xu khi đánh giá sản phẩm.
* Kế tiếp, ta nên xóa các sample mà khả năng cao ko là tiếng việt, vì sao ta làm bước này, đơn giản thôi đây là shopee việt nam, và các comment cố ý bằng tiếng anh, tiếng hàn, tiếng trung của các "thánh làm màu" sẽ là các noise sample khiến model ta bị giảm hiệu năng.
* Nhưng làm sao ta có thể thực hiện điều này, cách đơn giản nhất là ta có thể sử dụng các package như `textblob`, `googletrans`,... các package này chứa các function giúp ta detect language cho text, tuy nhiên hạn chế là chúng chỉ cho tối đa khoảng 200 request một ngày thôi, và số mẫu của chúng ta hiện tại là quá lớn. Ở đây ta có file `modules/dependencies/vocabulary.txt` chứa hơn 17000 từ đơn phổ biến của tiếng việt.
* Vậy cách đơn giản hơn là ta có thể xây dựng một dictionary chứa các từ đơn của tiếng việt, với mỗi comment, nếu số lượng từ ko tìm thấy trong dictionary này lớn hơn số từ được tìm thấy trong dictionary thì khả năng cao đây là một comment làm màu.
* Tuy nhiên, vẫn có một vài từ tiếng anh mà ta cần giữ lại như shipper, ta sẽ sử dụng package `enchant` để check một từ có phải là từ tiếng anh hay không.
  ```shell
  pip3 install pyenchant
  ```
* Ở các bước phía trên, ta đã đề cập đến việc xóa các từ bị dupplicate kí tự, ta sẽ thực hiện nó ở trong bước này.

In [82]:
!  pip3 install pyenchant

In [86]:
# hơn 17 ngàn từ đơn trong tiêng việt
vocabularies = Utils.buildDictionaryFromFile('./modules/dependencies/vocabulary.txt', True)
english_voca = enchant.Dict('en_US') # english if a word is english

In [87]:
reviews['normalize_comment'] = reviews['normalize_comment'].apply(lambda cmt: Processor.removeNoiseWord(cmt, vocabularies, english_voca))

reviews.tail()

,raw_comment,label,normalize_comment,emoji
270931,Hàng ok lắm có điều mùi nhựa nồng q...,1,hàng ok lắm có điều mùi nhựa nồng q...,
270932,Mua size L hơi to nhưng vẫn ok,1,mua size l hơi to nhưng vẫn ok,
270933,"Giá tốt. Chất lượng chắc chắn, thi...",1,giá tốt chất lượng chắc chắn thiê...,
270934,Sản phẩm tốt dày chất lượng size m ...,1,sản phẩm tốt dày chất lượng size mi...,
270935,"Đẹp lắm ạ\nCảm ơn shop, có điều bên ...",1,đẹp lắm ạ cảm ơn shop có điều bên ng...,


<hr>

* Tiếp theo ta sẽ remove stopword, chúng ta sẽ sử dụng stopword trong file `modules/dependencies/stopwords.txt`. Ta không nên sử dụng các stopword được build sẵn trên mạng nhất là cho tiếng việt, vì chưa chắc các từ này đã hợp với dữ liệu hiện tại của chúng ta.
* Ví dụ nhiều stopword set loại bỏ từ "**nhưng**", tuy nhiên từ này khả năng cao là quan trọng, giả sử ta có câu này:
  * _shop giao hàng chậm **nhưng** giao đúng hàng, ủng hộ shop_, thì nhờ từ **nhưng** này mà model ta có khả năng phân biệt được nó là positive hay negative.
* Ngoài ra với một file txt như vậy, ta có thể bổ sung stopword sau này.

In [95]:
stopwords = Utils.buildListFromFile("./modules/dependencies/stopwords.txt", encoding='utf-8')

In [96]:
reviews['normalize_comment'] = reviews['normalize_comment'].apply(lambda cmt: Processor.removeStopwords(cmt, stopwords))

reviews.tail()

,raw_comment,label,normalize_comment,emoji
270931,Hàng ok lắm có điều mùi nhựa nồng q...,1,hàng ok lắm có điều mùi nhựa nồng q...,
270932,Mua size L hơi to nhưng vẫn ok,1,mua size l hơi to nhưng vẫn ok,
270933,"Giá tốt. Chất lượng chắc chắn, thi...",1,giá tốt chất lượng chắc chắn thiê...,
270934,Sản phẩm tốt dày chất lượng size m ...,1,sản phẩm tốt dày chất lượng size nh...,
270935,"Đẹp lắm ạ\nCảm ơn shop, có điều bên ...",1,đẹp lắm cảm ơn có điều bên ngoài dí...,


<hr>

* Tiếp theo, ta loại bỏ các empty và duplicate `normalize_comment`.

In [97]:
reviews = Processor.removeEmptyOrDuplicateComment(reviews)

Processor.printAfterProcess(reviews)
reviews.tail()

Shape: (217218, 4)
1    210117
0      7101
Name: label, dtype: int64


,raw_comment,label,normalize_comment,emoji
217213,Hàng ok lắm có điều mùi nhựa nồng q...,1,hàng ok lắm có điều mùi nhựa nồng q...,
217214,Mua size L hơi to nhưng vẫn ok,1,mua size l hơi to nhưng vẫn ok,
217215,"Giá tốt. Chất lượng chắc chắn, thi...",1,giá tốt chất lượng chắc chắn thiê...,
217216,Sản phẩm tốt dày chất lượng size m ...,1,sản phẩm tốt dày chất lượng size nh...,
217217,"Đẹp lắm ạ\nCảm ơn shop, có điều bên ...",1,đẹp lắm cảm ơn có điều bên ngoài dí...,


<hr>

* Train test split, ta thấy rằng giữa hai nhóm positive và negative có chênh lệnh lớn, nên tập train data của ta sẽ bằng $0.8 * \min(\mathrm{size}(positive), \mathrm{size}(negative)) * 2$.

In [98]:
half_min_size = min(reviews['label'].value_counts())

half_min_size

7101

In [99]:
reviews_positive = reviews[reviews['label'] == 1]
reviews_negative = reviews[reviews['label'] == 0]

reviews_positive = shuffle(reviews_positive)
reviews_positive = reviews_positive.reset_index(drop=True)

In [100]:
positive_index = random.sample(range(0, reviews_positive.shape[0]), half_min_size)

positive_index[:10]

[9180, 192424, 74152, 113600, 193900, 160773, 175237, 22706, 93840, 161098]

In [101]:
reviews_positive2 = reviews_positive.iloc[positive_index,:]

reviews_positive2.head()

,raw_comment,label,normalize_comment,emoji
9180,Giao đúng mẫu đúng số lượng đã đặt...,1,giao đúng mẫu đúng số lượng đặt cha...,
192424,Sản phẩm chất lượng mặc dù có hơi...,1,sản phẩm chất lượng mặc dù có hơi...,
74152,Hàng đẹp giá cả phải chăng chất vải k...,1,hàng đẹp giá cả phải chăng chất vải k...,
113600,"Hàng đẹp, chất lượng lắm ạ. Cực ki...",1,hàng đẹp chất lượng lắm cực kì ưn...,😍
193900,"giao đầy đủ số lượng, chất lượng ...",1,giao đầy đủ số lượng chất lượng v...,


* Đây là tập data mà hai nhóm positive và negative cân bằng nhau

In [102]:
normalize_reviews = pd.concat([reviews_negative, reviews_positive2], axis=0)
normalize_reviews = normalize_reviews.reset_index(drop=True)

Processor.printAfterProcess(normalize_reviews)
normalize_reviews.head()

Shape: (14202, 4)
1    7101
0    7101
Name: label, dtype: int64


,raw_comment,label,normalize_comment,emoji
0,Quần giặt xong co ngắn mất 5 phân. Ne...,0,quần giặt xong co ngắn mất phân nên ...,
1,Vải áo mặc nóng \nCòn vải quần dễ ...,0,vải áo mặc nóng còn vải quần dễ xù...,❤️
2,"Đóng gói cẩn thận, giao đủ số lượn...",0,đóng gói cẩn thận giao đủ số lượng...,
3,Quần quá to mọi người ạ còn dài nx ...,0,quần quá to mọi người còn dài mặc d...,
4,Vải thừa hơi nhiều\nGiao hàng nhanh .!\...,0,vải thừa hơi nhiều giao hàng nhanh thủ...,


* Ghi ra file

In [103]:
normalize_reviews.to_csv("./data/normalize_reviews.csv", index=False)

* Bây giờ ta sẽ ghi phần bù còn lại của `review_positive` vào file, ta có thể dùng nó cho việc evaluate model sau này.

In [104]:
reviews_positive3 = reviews_positive[~reviews_positive.index.isin(positive_index)]
reviews_positive3 = reviews_positive3.reset_index(drop=True)

reviews_positive3.head()

,raw_comment,label,normalize_comment,emoji
0,"mình săn sale nên rẻ lắm\nvớ xinh, xị...",1,săn sale nên rẻ lắm vớ xinh xịn lắm ...,🤤
1,Chất mát. Giá cả phù hợp với chất...,1,chất mát giá cả phù hợp chất lượn...,❤️
2,Móng giống hình\nNhưng chai keo giống k...,1,móng giống hình nhưng chai keo giống ke...,
3,"Shop có tâm, quần đẹp, áo đẹp lắm ạ ...",1,có tâm quần đẹp áo đẹp lắm,
4,Áo đẹppp size M m6 mặc thoải mái nha mng...,1,áo đẹp size mình mặc thoải mái mọi ngu...,


In [105]:
reviews_positive3.to_csv("./data/complement_positive_reviews.csv", index=False)